<a href="https://colab.research.google.com/github/BERENZ/temp-empl-polit-pref/blob/main/1-setup-and-data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [1]:
install.packages("tidyverse")
install.packages("data.table")
install.packages("lmtest")
install.packages("sandwich")
install.packages("survey")
install.packages("broom")
install.packages("stringi")
install.packages("validate")
install.packages("VIM")
install.packages("haven")
install.packages("lme4")
install.packages("lmerTest")
install.packages("bife")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘settings’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘matrixStats’, ‘RcppArmadillo’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘

# Structure of the project

Create structure of the project and download required data

In [2]:
## structure of the project
if (!dir.exists("codes")) dir.create("codes")
if (!dir.exists("results")) dir.create("results")
if (!dir.exists("data")) dir.create("data")
if (!dir.exists("data-raw")) dir.create("data-raw")

## download data from Diagnoza.com
if (!file.exists("data-raw/hh_data.sav")) {
  download.file("http://diagnoza.com/pliki/bazy_danych/2015/ds_0_15_GD_14112015.sav", destfile = "data-raw/hh_data.sav") 
  gosp <- haven::read_spss("data-raw/hh_data.sav", user_na = TRUE)
  saveRDS(gosp, file = "data/hh_data.RDS")
}
if (!file.exists("data-raw/ind_data.sav")) {
  download.file("http://diagnoza.com/pliki/bazy_danych/2015/ds_0_15_ind_14112015.sav", destfile = "data-raw/ind_data.sav") 
  osob <- haven::read_spss("data-raw/ind_data.sav", user_na = TRUE)
  saveRDS(osob, file = "data/ind_data.RDS")
}

# Data analysis 

## Packages and data

In [ ]:
library(tidyverse)
library(data.table)
library(lmtest)
library(sandwich)
library(survey)
library(broom)
library(stringi)
library(Matrix)
library(validate)
library(VIM)
library(haven)

In [4]:
gosp <- readRDS("data/hh_data.RDS")
osob <- readRDS("data/ind_data.RDS")

In [5]:
ls()

[1] "gosp" "osob"

## Read data 

Variables used in the study

+ Abstention and Gov_support -- fp106, GP98, HP101
+ Gov_respons -- ep57, ep58_1, fp59, fp60_1, GP57, GP58_1, HP60, HP61_1
+ Inc_equal -- ep56_26, fp58_21, GP54_18, HP57_14
+ Fixed_temp, Other_temp, Permanent, Init_temp  -- ed05, fD5, GD5, HD5 
+ Inc_instab_often, Inc_instab_some, Inc_instab_never -- ep14, fp14, GP13, HP13
+ Sat_fin_neg, Sat_fin_neut, Sat_fin_pos -- ep64_2, fp63_2, GP62_02, HP63_02
+ Sat_work_neg, Sat_work_neut, Sat_work_pos -- ep64_17, fp63_13, GP62_13, HP63_13
+ Public -- status9_2009, status9_2011, status9_2013, status9_2015
+ Female - GENDER
+ Tert_post_edu, Up_second_edu, Low_voc_edu, Primary_edu -- educ4_2009, educ4_2011, educ4_2013, educ4_2015
+ Age -- AGE2009, AGE2011, AGE2013, AGE2015
+ WAVE -- F2009, F2011, F2013, F2015

**new variables**

+ In_education -- ec15, fc19, GC19, HC19
+ Income_source -- ec39, fc37, GC32, HC32
+ Occupation -- ed18_22, FD18_19, GD18_19, HD18_19


In [6]:
gosp  %>%
  select(gosp_id = numer_gd,
         f2000:f2015,
         L_OSOB_2000:L_OSOB_2015,
         WAGA_GD_2000:WAGA_GD_2015,
         klm = KLASA_MIEJSCOWOSCI,
         woj = WOJEWODZTWO,
         pow = POWIAT) %>%
  mutate(gosp_id = as.integer(gosp_id)) %>%
  set_names(tolower(names(.))) -> gosp_subset
head(gosp_subset)

gosp_id,f2000,f2003,f2005,f2007,f2009,f2011,f2013,f2015,l_osob_2000,⋯,waga_gd_2003,waga_gd_2005,waga_gd_2007,waga_gd_2009,waga_gd_2011,waga_gd_2013,waga_gd_2015,klm,woj,pow
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,⋯,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl>
1,1,NA,NA,NA,NA,NA,NA,NA,6,⋯,0.000000,0.000000,0.0000000,0.000000,0.000000,NA,NA,5,2,4
2,1,1,1,1,1,1,1,1,2,⋯,1.251585,1.112111,0.3304216,0.350743,0.318502,0.458628,0.201015,6,2,4
3,1,1,1,NA,NA,NA,NA,NA,6,⋯,0.640080,0.666600,0.0000000,0.000000,0.000000,NA,NA,6,2,13
4,1,1,1,1,1,NA,NA,NA,2,⋯,1.251585,1.112111,1.5144856,1.297467,0.000000,NA,NA,5,2,13
5,1,NA,NA,NA,NA,NA,NA,NA,2,⋯,0.000000,0.000000,0.0000000,0.000000,0.000000,NA,NA,4,2,1
6,1,1,1,1,1,NA,NA,NA,3,⋯,1.083564,1.112111,1.7968025,1.927316,0.000000,NA,NA,4,2,14


Cleaning the data

- in 2005 -- hh no.: 2657 not in a wave but weight  > 0
- w 2007 -- hh no.: 60,2379,10170  not in a wave but weight   > 0
- w 2011 -- hh no.: c(2163L, 2165L, 2166L, 8176L, 8181L, 10694L, 31274L, 31290L,   31385L, 31387L, 38365L, 40412L, 40674L, 41569L, 45302L, 45722L,  45900L, 45901L, 47133L, 47349L, 50758L, 53883L, 54198L, 54199L,  54271L, 54272L, 54273L, 54348L, 54366L, 54370L, 54380L, 54707L) weight > 0
- w 2013 -- hh no.: c(7337L, 27019L, 39798L, 46010L, 46221L, 51932L, 54748L, 100317L,  101345L)
- w 2015 -- hh no.: c(2049L, 41576L, 51287L, 51513L)

Differences between weights and number of observations
- 2000 - yes, yes
- 2003 - no (3961), no -> after changes yes
- 2005 - no (3851), no -> yes
- 2007 - yes, yes -> nie (wagi tez sie nie sumowly, 5532) 
- 2009 - yes, yes -> yes
- 2011 - no (12381), no -> yes 
- 2013 - no (12352), no  -> yes
- 2015 - no (11740), no -> nie (dwie za dużo)

Numer of persons but weight is NA

-- 2003 - 4330,
-- 2005 -- 31 hh
-- 2007 -- 30 hh
-- 2011 -- 10 hh 

In [7]:
gosp_subset %>% 
  mutate(waga_gd_2000 = as.numeric(unclass(waga_gd_2000)),
         waga_gd_2003 = as.numeric(unclass(waga_gd_2003)),
         waga_gd_2005 = as.numeric(unclass(waga_gd_2005)),
         waga_gd_2007 = as.numeric(unclass(waga_gd_2007)),
         waga_gd_2009 = as.numeric(unclass(waga_gd_2009)),
         waga_gd_2011 = as.numeric(unclass(waga_gd_2011)),
         waga_gd_2013 = as.numeric(unclass(waga_gd_2013)),
         waga_gd_2015 = as.numeric(unclass(waga_gd_2015)),
         f2000 = (as.numeric(waga_gd_2000) > 0)*1,
         f2003 = (as.numeric(waga_gd_2003) > 0)*1,
         f2005 = (as.numeric(waga_gd_2005) > 0)*1,
         f2007 = (as.numeric(waga_gd_2007) > 0)*1,
         f2009 = (as.numeric(waga_gd_2009) > 0)*1,
         f2011 = (as.numeric(waga_gd_2011) > 0)*1,
         f2013 = (as.numeric(waga_gd_2013) > 0)*1,
         f2015 = (as.numeric(waga_gd_2015) > 0)*1,
         waga_gd_2000 = ifelse(f2000 == 0, NA,waga_gd_2000),
         waga_gd_2003 = ifelse(f2003 == 0, NA,waga_gd_2003),
         waga_gd_2005 = ifelse(f2005 == 0, NA,waga_gd_2005),
         waga_gd_2007 = ifelse(f2007 == 0, NA,waga_gd_2007),
         waga_gd_2009 = ifelse(f2009 == 0, NA,waga_gd_2009),
         waga_gd_2011 = ifelse(f2011 == 0, NA,waga_gd_2011),
         waga_gd_2013 = ifelse(f2013 == 0, NA,waga_gd_2013),
         waga_gd_2015 = ifelse(f2015 == 0, NA,waga_gd_2015)) %>%
  mutate_at(vars(f2000:f2015), list(~ifelse(is.na(.),0,.))) %>%
  mutate_at(vars(waga_gd_2000:waga_gd_2015),
            list(~. / mean(., na.rm=T))) %>%
  gather(fala, fala_vals, f2000:f2015) %>%
  gather(osoby, osoby_vals, l_osob_2000:l_osob_2015) %>%
  gather(waga, waga_vals, waga_gd_2000:waga_gd_2015) %>%
  filter(fala_vals == 1) %>%
  mutate_at(vars(fala,osoby,waga), list(~stri_extract_first_regex(., '\\d{4}'))) %>%
  filter(fala == osoby, waga == fala, osoby == fala) %>%
  select(-fala_vals,-osoby, -waga) %>%
  rename(l_osob = osoby_vals, waga = waga_vals) -> gosp_cleaned

head(gosp_cleaned)

Warning message:
“attributes are not identical across measure variables;
they will be dropped”


gosp_id,klm,woj,pow,fala,l_osob,waga
<int>,<dbl+lbl>,<dbl+lbl>,<dbl>,<chr>,<dbl>,<dbl>
1,5,2,4,2000,6,0.8433150
2,6,2,4,2000,2,1.2784563
3,6,2,13,2000,6,1.0417421
4,5,2,13,2000,2,1.2784563
5,4,2,1,2000,2,1.2784563
6,4,2,14,2000,3,0.4906076


In [8]:
saveRDS(object = gosp_cleaned, file = 'data/hh_subset.rds')

In [9]:
osob %>%
  select(gosp_id = Nuner_GD, 
         person_num = NUMER_OSOBY,
         person_id = numer150730,
         sex = PLEC_ALL,
         wiek2000:wiek2015,
         woj = WOJEWODZTWO,
         klm = KLASA_MIEJSCOWOSCI,
         WAGA_2000_IND:WAGA_2015_IND, ## waga przypisana do osob 
         #WAGA_2000_OSOBY:WAGA_2015_OSOBY, ## waga przypisana do osob, które wypełniły kwestionariusz indyw
         F2000:F2015, ## informacja czy dana osoba brała udział w określonej fali
         INDYWIDUALNI_2000:INDYWIDUALNI_2015, ## czy braki udział w wypelnieniu ankiety indywidualnej
         eduk4_2000:eduk4_2015, ## edukacja
         status9_2000:status9_2015,
         ## grupy zawodowe
         zawod10_2009 = E_10GRUP_ZAWODOWYCH_OGOLEM, zawod10_2011 = F_10GRUP_ZAWODOWYCH_OGOLEM,
         zawod10_2013 = G_10GRUP_ZAWODOWYCH_OGOLEM, zawod10_2015 = H_10GRUP_ZAWODOWYCH_OGOLEM,
         ## dochody (teraz)
         doch_2009 = ep66, doch_2011 = fp65, doch_2013 = GP64, doch_2015 = HP65, 
         ## dochody za 2 lata
         doch2_2009 = ep67, doch2_2011 = fp66, doch2_2013 = GP65, doch2_2015 = HP66, 
         ## zadowolenie z pracy (dla pracujacych)
         zadprac_2000 = ap77_20, zadprac_2003 = bp67_17, zadprac_2005 = cp67_17, zadprac_2007 = dp61_17,
         zadprac_2009 = ep64_17,  zadprac_2011 = fp63_13, zadprac_2013 = GP62_13,  zadprac_2015 = HP63_13, 
         ## zadowolenie z sytuacji finansowej
         zadfin_2000 = ap77_2, zadfin_2003 = bp67_2, zadfin_2005 = cp67_2, zadfin_2007 = dp61_2,
         zadfin_2009 = ep64_2, zadfin_2011 = fp63_2, zadfin_2013 = GP62_02, zadfin_2015 = HP63_02,
         ## niestabilność dochodów
         niestab_2000 = ap18, niestab_2003 = bp17, niestab_2005 = cp17, niestab_2007 = dp17,
         niestab_2009 = ep14, niestab_2011 = fp14, niestab_2013 = GP13, niestab_2015 = HP13,
         ## rozdzaj pracy (wcześniej było pytanie czy to praca w pełnym wymiarze)
         praca_2007 = dd5, praca_2009 = ed05, praca_2011 = fD5, praca_2013 = GD5, praca_2015 = HD5,
         ## równosć dochodów
         rowndoch_2005 = cp54_12, rowndoch_2007 = dp50_14,
         rowndoch_2009 = ep56_26, rowndoch_2011 = fp58_21, rowndoch_2013 = GP54_18, rowndoch_2015 = HP57_14,
         ## partie (nie ma wcześniejszych?)
         partie_2011 = fp106, partie_2013 = GP98, partie_2015 = HP101,
         ## sejm (wcześniejszych nie ma? )
         sejm_2009 = ep101_2, sejm_2011 = fp98_3, sejm_2013 = GP104_03, sejm_2015 = HP105_03,
         ## deomkaracja,
         demo_2003 = bp99, demo_2005 = cp102, demo_2007 = dp71,
         demo_2009 = ep102, demo_2011 = fp64, demo_2013 = GP63, demo_2015 = HP64,
         ## bezrobocie
         bezr_2009 = ed21, bezr_2011 = fD21, bezr_2013 = GD21, bezr_2015 = HD21,
         ## zarejstrowany
         bezrrej_2009 = ed20, bezrrej_2011 = fD20, bezrrej_2013 = GD20, bezrrej_2015 = HD20,
         ## udane  życie
         udane_2000 = ap75, udane_2003 = bp68, udane_2005 = cp64, udane_2007 = dp59, 
         udane_2009 = ep57, udane_2011 = fp59, udane_2013 = GP57, udane_2015 = HP60, 
         ## udane dzięki rządowi (w 2005 nie ma tego pytana)
         udane2_2000 = ap76_1,  udane2_2003 = bp69_1, udane2_2007 = dp60_1, 
         udane2_2009 = ep58_1,  udane2_2011 = fp60_1, udane2_2013 = GP58_1, udane2_2015 = HP61_1,
         ## inni ludzie 
         locinni_2009 = ep58_3,  locinni_2011 = fp60_3, locinni_2013 = GP58_3, locinni_2015 = HP61_3,
         ## los
         loclos_2009 = ep58_4,  loclos_2011 = fp60_4, loclos_2013 = GP58_4, loclos_2015 = HP61_4,
         ## ode nich samych
         losint_2009 = ep58_2,  losint_2011 = fp60_2, losint_2013 = GP58_2, losint_2015 = HP61_2,
         ## poziom wyksz
         pozwyk_2009 = poziom_wykształcenia_2009, pozwyk_2011 = poziom_wykształcenia_2011,
         pozwyk_2013 = poziom_wykształcenia_2013, pozwyk_2015 = poziom_wykształcenia_2015,
         ## new variables
         ### #in education
         inedu_2009 = ec15, inedu_2011 = fC19, inedu_2013 = GC19, inedu_2015 = HC19,
         ## income source
         incsource_2009 = ec39, incsource_2011 = fC37, incsource_2013 = GC32, incsource_2015 = HC32,
         ## occupation
         occup_2009 = e_zawod_ogolem, occup_2011 = F_ZAWOD_OGOLEM_SKR, occup_2013 = G_ZAWOD_OGOLEM_SKR, 
         occup_2015 = H_ZAWOD_OGOLEM_SKR
         )  %>%
  mutate_at(vars(F2000:F2015), list(~ifelse(is.na(.), 0, .)))  %>%
  mutate_at(vars(INDYWIDUALNI_2000:INDYWIDUALNI_2015), list(~ifelse(is.na(.), 0, .)))  %>%
  mutate_at(vars(WAGA_2000_IND:WAGA_2015_IND), list(~ifelse(is.na(.), 0, .)))  %>%
  rename(f_2000 = F2000 , f_2003= F2003 ,  f_2005=F2005,  f_2007 = F2007,
         f_2009 = F2009 , f_2011 = F2011, f_2013 = F2013, f_2015 = F2015,
         wiek_2000 = wiek2000 , wiek_2003 = wiek2003 ,  wiek_2005=wiek2005,  wiek_2007 = wiek2007,
         wiek_2009 = wiek2009 , wiek_2011 = wiek2011, wiek_2013 = wiek2013, wiek_2015 = wiek2015) -> osoby_df


In [10]:
check_that(osoby_df,
           if (F2005 == 1) INDYWIDUALNI_2005 > 0 ,
           if (F2007 == 1) INDYWIDUALNI_2007 > 0 ,
           if (F2009 == 1) INDYWIDUALNI_2009 > 0 ,
           if (F2011 == 1) INDYWIDUALNI_2011 > 0 ,
           if (F2013 == 1) INDYWIDUALNI_2013 > 0 ,
           if (F2015 == 1) INDYWIDUALNI_2015 > 0 ,
           if (F2005 == 1 & wiek2005 >=16) INDYWIDUALNI_2005 > 0 ,
           if (F2007 == 1 & wiek2007 >=16) INDYWIDUALNI_2007 > 0 ,
           if (F2009 == 1 & wiek2009 >=16) INDYWIDUALNI_2009 > 0 ,
           if (F2011 == 1 & wiek2011 >=16) INDYWIDUALNI_2011 > 0 ,
           if (F2013 == 1 & wiek2013 >=16) INDYWIDUALNI_2013 > 0 ,
           if (F2015 == 1 & wiek2015 >=16) INDYWIDUALNI_2015 > 0 ,
           # if (INDYWIDUALNI_2005 == 1) WAGA_2005_IND > 0 ,
           # if (INDYWIDUALNI_2007 == 1) WAGA_2007_IND > 0 ,
           # if (INDYWIDUALNI_2009 == 1) WAGA_2009_IND > 0 ,
           # if (INDYWIDUALNI_2011 == 1) WAGA_2011_IND > 0 ,
           # if (INDYWIDUALNI_2013 == 1) WAGA_2013_IND > 0 ,
           # if (INDYWIDUALNI_2015 == 1) WAGA_2015_IND > 0 ,
           if (INDYWIDUALNI_2015 == 1) partie_2015 > 0) %>%
  summary()

name,items,passes,fails,nNA,error,warning,expression
<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<chr>
V01,0,0,0,0,TRUE,FALSE,!(abs(F2005 - 1) < 1e-08) | (INDYWIDUALNI_2005 > 0)
V02,0,0,0,0,TRUE,FALSE,!(abs(F2007 - 1) < 1e-08) | (INDYWIDUALNI_2007 > 0)
V03,0,0,0,0,TRUE,FALSE,!(abs(F2009 - 1) < 1e-08) | (INDYWIDUALNI_2009 > 0)
V04,0,0,0,0,TRUE,FALSE,!(abs(F2011 - 1) < 1e-08) | (INDYWIDUALNI_2011 > 0)
V05,0,0,0,0,TRUE,FALSE,!(abs(F2013 - 1) < 1e-08) | (INDYWIDUALNI_2013 > 0)
V06,0,0,0,0,TRUE,FALSE,!(abs(F2015 - 1) < 1e-08) | (INDYWIDUALNI_2015 > 0)
V07,0,0,0,0,TRUE,FALSE,!(abs(F2005 - 1) < 1e-08 & wiek2005 >= 16) | (INDYWIDUALNI_2005 > 0)
V08,0,0,0,0,TRUE,FALSE,!(abs(F2007 - 1) < 1e-08 & wiek2007 >= 16) | (INDYWIDUALNI_2007 > 0)
V09,0,0,0,0,TRUE,FALSE,!(abs(F2009 - 1) < 1e-08 & wiek2009 >= 16) | (INDYWIDUALNI_2009 > 0)


In [11]:
osoby_df %>%
  mutate(WAGA_2005_IND = ifelse(INDYWIDUALNI_2005 == 1, WAGA_2005_IND, 0),
         WAGA_2007_IND = ifelse(INDYWIDUALNI_2007 == 1, WAGA_2007_IND, 0),
         WAGA_2009_IND = ifelse(INDYWIDUALNI_2009 == 1, WAGA_2009_IND, 0),
         WAGA_2011_IND = ifelse(INDYWIDUALNI_2011 == 1, WAGA_2011_IND, 0),
         WAGA_2013_IND = ifelse(INDYWIDUALNI_2013 == 1, WAGA_2013_IND, 0),
         WAGA_2015_IND = ifelse(INDYWIDUALNI_2015 == 1, WAGA_2015_IND, 0)) %>% 
  ## usuwam błędne wartości
  mutate_at(vars(zadfin_2015, partie_2015, niestab_2015, zadprac_2015, udane_2015, udane2_2015, rowndoch_2015,
                 locinni_2015, loclos_2015, losint_2015, pozwyk_2015, inedu_2015, incsource_2015, occup_2015),
            list(~if_else(INDYWIDUALNI_2015 == 1, ., -9))) %>%
  mutate_at(vars(zadfin_2013, partie_2013, niestab_2013, zadprac_2013, udane_2013, udane2_2013, rowndoch_2013,
                 locinni_2013, loclos_2013, losint_2013, pozwyk_2013, inedu_2013, incsource_2013, occup_2013),
            list(~if_else(INDYWIDUALNI_2013 == 1, ., -9))) %>%
  mutate_at(vars(zadfin_2011, partie_2011, niestab_2011, zadprac_2011, udane_2011, udane2_2011, rowndoch_2011,
                 locinni_2011, loclos_2011, losint_2011, pozwyk_2011, inedu_2011, incsource_2011, occup_2011),
            list(~if_else(INDYWIDUALNI_2011 == 1, ., -9))) %>%
  mutate_at(vars(zadfin_2009, niestab_2009, zadprac_2009, udane_2009, udane2_2009, rowndoch_2009,
                 locinni_2009, loclos_2009, losint_2009, pozwyk_2009, inedu_2009, incsource_2009, occup_2009),
            list(~if_else(INDYWIDUALNI_2009 == 1, ., -9)))  -> osoby_df

In [ ]:
osoby_df %>%
  gather(vars, vals, -gosp_id, -person_id, -sex, -woj, -klm, -person_num) %>%
  separate(vars, c('zm','rok'), sep = "_") %>% 
  spread(zm, vals) %>%
  filter(rok %in% 2009:2015, 
         f == 1, INDYWIDUALNI == 1, 
         WAGA > 0, wiek >= 16) %>% 
  select(gosp_id:rok, wiek, WAGA, everything()) %>%
  setNames(tolower(names(.))) %>%
  select(-f, -indywidualni) %>%
  left_join(gosp_cleaned %>% 
              rename(waga_gosp = waga,
                     klm_gosp = klm,
                     woj_gosp = woj,
                     pow_gosp  = pow,
                     rok = fala,
                     l_osob_gosp = l_osob)) -> os_df 
            
 #
os_df  %>%
  mutate(flag = !(gosp_id == 47482 & unclass(klm_gosp) == 4)) %>%
  filter(flag) -> os_df

Warning message:
“attributes are not identical across measure variables;
they will be dropped”


In [ ]:
os_df %>%
  filter(!praca %in% c(3, 4, 5, NA, -8), rok %in% 2009:2015, !is.na(woj), zadfin != 7, niestab != 4) %>%
  #mutate_at(vars(klm, klm_gosp, rok, sex, woj, woj_gosp), as.numeric) %>%
  mutate_all(funs(ifelse(unclass(.) <0 , NA, unclass(.)))) %>%
  mutate(bezrej = replace_na(bezrrej, 0),
         bezr = replace_na(bezr, 0)) -> data_for_model ## nie dotyczy -- braki w dochodzie (brak dochodów  
  
data_for_model %>% head()

In [ ]:
data_for_model_imp <- data_for_model %>%
  mutate_at(vars(sex,woj, klm, demo, eduk4:pow_gosp), factor)

In [ ]:
VIM::aggr(data_for_model_imp)

In [ ]:
cols_names <- c("sex","wiek", "rok", "woj", "klm", "demo", "eduk4", "niestab", "partie", "praca",
                "rowndoch", "sejm", "status9", "udane", "udane2", "zadfin", "zadprac",
                "locinni", "loclos", "losint", "incsource", "inedu")
data_for_model_imp <- kNN(data = data_for_model_imp, 
                          variable = cols_names, 
                          dist_var = cols_names, 
                          k = 1, 
                          trace = TRUE)

In [ ]:
data_for_model_imp <- data_for_model_imp %>%
  select(-ends_with("imp")) %>%
  mutate(gov_support = case_when(rok == 2009 ~ NA_real_, 
                                 rok == 2011 & partie == 5 ~ 1,
                                 rok == 2013 & partie == 7 ~ 1,
                                 rok == 2015 & partie == 1 ~ 1,
                                 TRUE ~ 0 ),
         inq_equal = case_when(rok == 2009 & rowndoch %in% 1:3 ~ 1,
                               rok == 2011 & rowndoch %in% 1:3 ~ 1,
                               rok == 2013 & rowndoch %in% 1:3 ~ 1,
                               rok == 2015 & rowndoch %in% 1:3 ~ 1,
                               TRUE ~ 0 ),
         gov_respons = udane == 2 & udane2 == 1,
         public = as.numeric(status9 == 1),
         gov_respons = as.numeric(gov_respons),
         praca2 = case_when(praca == 1 ~ "Fixed_temp",
                            praca == 2 ~ "Permanent", 
                            TRUE ~ "Other_temp"),
         klm2 = case_when(klm == 6 ~ 1,
                          klm == 1 ~ 2,
                          TRUE ~ 3),
         klm2 = factor(klm2),
         ex_loc_control = udane2 == 1 | locinni == 1 | loclos == 1,
         in_loc_control = losint == 1,
         wyzsze = pozwyk %in% c(10,11,12),
         wyksz_5 = case_when(pozwyk %in% c(60, 70) ~ "basic or none",
                             pozwyk %in% c(51) ~ "gminazj",
                             pozwyk %in% c(50) ~ "vocational",
                             pozwyk %in% c(20, 30, 40) ~ "second",
                             pozwyk %in% c(10, 11, 20, 30) ~ "higher"),
         praca2 = factor(praca2),  praca2 = relevel(praca2, ref = "Permanent"),
         niestab = factor(niestab, 1:3, c("often", "some", "never")),
         zadfin2 = case_when(zadfin %in% 1:2 ~ "pos", zadfin %in% 3:4 ~ "neu", zadfin %in% 5:6 ~ "neg"),
         zadfin2 = factor(zadfin2),
         zadprac2 = case_when(zadprac %in% 1:2 ~ "pos", zadprac %in% 3:4 ~ "neu", zadprac %in% 5:6 ~ "neg"),
         zadprac2 = factor(zadprac2),
         demo2 = demo == 1,
         inedu2 = case_when(inedu == 2 ~ 1, inedu == 3 ~ 2, inedu %in% c(4,5) ~ 3, inedu %in% c(1,8) ~ 4),
         incsource2 = case_when(incsource %in% c(11, 12, 13, 14, 15, 16, 17, 18, 19, 20) ~ 1,
                                incsource %in% c(21, 22, 37) ~ 2,
                                incsource %in% c(23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34) ~ 3,
                                incsource %in% c(35, 36, 38, 39, 40, 41, 42) ~ 4,
                                incsource %in% c(43) ~ 5))


In [ ]:
saveRDS(data_for_model_imp, file = "data/data_for_model_2009_2015.rds")